In [2]:
import yaml

In [3]:
yaml_args= yaml.load(open('SA-config/en_ewt-pos-corrupted0-rank1000-1hid-BERT-base-1-SA-50.yaml'))

/home/felix/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [9]:



maximum_rank = 1000 # [1000,300,50,10]
model_layer = 1 # [1,3,6,9,12]

dim_num_list = [768, 400, 300, 200, 150, 125, 100, 90, 80, 70, 60, 50, 40, 30, 20,18,16,14,12,10,9,8,7,6,5,4,3,2,1]
corrupted_token_percent = 0.0 # [0.0, 1.0]
corr_compute_method = 'average' # ['average', 'weighted_average', 'max', 'random']
dim_file = 'SA-dim-files/{}_corr_dim.tsv'.format(corr_compute_method) 
# ['SA-dim-files/average_corr_dim.tsv','SA-dim-files/weighted_average_corr_dim.tsv','SA-dim-files/max_corr_dim.tsv']

hidden_layer=1
exp_group_name='layer-{}-maximum_rank-{}'.format(model_layer,maximum_rank)
results_file = 'hid-{}-results-{}.csv'.format(hidden_layer,exp_group_name)
bash_file = 'scripts/hid-{}-exp-{}.sh'.format(hidden_layer,exp_group_name)
comment = 'hid-{}-exp-{}'.format(hidden_layer,exp_group_name)

def gen_bash_file(maximum_rank,dim_num_list,comment,corrupted_token_percent,dim_file,model_layer,results_file,bash_file,corr_compute_method,hidden_layers,CUDA_VISIBLE_DEVICES,do_sub_dim_rest=False):

    yaml_args= yaml.load(open('SA-config/en_ewt-pos-corrupted0-rank1000-1hid-BERT-base-1-SA-50.yaml'))
    

    
    for dim_num in dim_num_list:
        yaml_args['dataset']['sub_dim']['dim_num']=dim_num
        
        if do_sub_dim_rest:
            yaml_args['dataset']['sub_dim']['do_sub_dim_rest']=True
            yaml_args['dataset']['sub_dim']['do_sub_dim']=False
            yaml_args['model']['hidden_dim']=768-dim_num
        else:
            yaml_args['dataset']['sub_dim']['do_sub_dim_rest']=False
            yaml_args['dataset']['sub_dim']['do_sub_dim']=True
            yaml_args['model']['hidden_dim']=dim_num
        
        yaml_args['model']['model_layer']=model_layer-1

        yaml_args['dataset']['sub_dim']['dim_file']=dim_file
        yaml_args['dataset']['batch_size']=200
        
        
        yaml_args['probe']['maximum_rank']=maximum_rank
        yaml_args['probe']['misc']['corrupted_token_percent']=corrupted_token_percent
        yaml_args['probe']['hidden_layers']=hidden_layers        
        yaml_args['reporting']['comment']=comment
        yaml_args['reporting']['results_file']=results_file

        
        out_file='SA-config/en_ewt-pos-corrupted{}-rank{}-{}-hid-BERT-base-layer-{}-SA-{}-corr-{}.yaml'.format(int(corrupted_token_percent),
                                                                                                       maximum_rank,
                                                                                                               hidden_layers,
                                                                                                             model_layer,
                                                                                                       dim_num,
                                                                                                       corr_compute_method)
        with open(out_file, 'w') as file:
            documents = yaml.dump(yaml_args, file)
        with open(bash_file, 'a') as fout:
            fout.write('CUDA_VISIBLE_DEVICES={} python control-tasks/run_experiment.py '.format(CUDA_VISIBLE_DEVICES)+out_file+'\n')
        


In [18]:
# for hidden_layers in [0,2]:
hidden_layers=1
#     for maximum_rank in [300,50,1000,10]:
CUDA_VISIBLE_DEVICES=3
do_sub_dim_rest=True
name_prefix='do_sub_dim_rest'

for maximum_rank in [300]:

    for model_layer in  [1,3,6,9,12]:
#         for corr_compute_method in ['average', 'weighted_average', 'max', 'random']:
        for corr_compute_method in ['average','random']:

            for corrupted_token_percent in [0.0,1.0]:
                if corrupted_token_percent==1.0 and corr_compute_method!='random':
                    continue

#                 dim_num_list = [768, 400, 300, 200, 150, 125, 100, 90, 80, 70, 60, 50, 40, 30, 20,18,16,14,12,10,9,8,7,6,5,4,3,2,1]
                dim_num_list = [400, 300, 200, 150, 125, 100, 90, 80, 70, 60, 50, 40, 30, 20,18,16,14,12,10,9,8,7,6,5,4,3,2,1]
                
#                 corrupted_token_percent = 0.0 # [0.0, 1.0]
                if corr_compute_method!='random':
                    dim_file = 'SA-dim-files/{}_corr_dim_layer_{}.tsv'.format(corr_compute_method,model_layer) 
                else:
                    dim_file = 'SA-dim-files/{}_corr_dim.tsv'.format(corr_compute_method) 
                # ['SA-dim-files/average_corr_dim.tsv','SA-dim-files/weighted_average_corr_dim.tsv','SA-dim-files/max_corr_dim.tsv']

#                     hidden_layer=1
                exp_group_name='{}-layer-{}-maximum_rank-{}'.format(name_prefix,model_layer,maximum_rank)
                results_file = '{}-hid-{}-results-{}.csv'.format(name_prefix,hidden_layers,exp_group_name)
                bash_file = 'scripts/{}-hid-{}-exp-{}.sh'.format(name_prefix,hidden_layers,exp_group_name)
                comment = '{}-hid-{}-exp-{}'.format(name_prefix,hidden_layers,exp_group_name)
                gen_bash_file(maximum_rank,dim_num_list,comment,corrupted_token_percent,dim_file,model_layer,results_file,bash_file,corr_compute_method,hidden_layers,CUDA_VISIBLE_DEVICES,do_sub_dim_rest)
        print(bash_file)



/home/felix/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-1-maximum_rank-300.sh
scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-3-maximum_rank-300.sh
scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-6-maximum_rank-300.sh
scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-9-maximum_rank-300.sh
scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-12-maximum_rank-300.sh


In [ ]:
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-1-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-3-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-12-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-6-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-9-maximum_rank-300.sh

bash scripts/do_sub_dim_rest-hid-0-exp-do_sub_dim_rest-layer-1-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-0-exp-do_sub_dim_rest-layer-3-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-0-exp-do_sub_dim_rest-layer-12-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-0-exp-do_sub_dim_rest-layer-6-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-0-exp-do_sub_dim_rest-layer-9-maximum_rank-300.sh

bash scripts/do_sub_dim_rest-hid-2-exp-do_sub_dim_rest-layer-1-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-2-exp-do_sub_dim_rest-layer-3-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-2-exp-do_sub_dim_rest-layer-12-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-2-exp-do_sub_dim_rest-layer-6-maximum_rank-300.sh
bash scripts/do_sub_dim_rest-hid-2-exp-do_sub_dim_rest-layer-9-maximum_rank-300.sh

bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-1-maximum_rank-50.sh
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-3-maximum_rank-50.sh
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-12-maximum_rank-50.sh
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-6-maximum_rank-50.sh
bash scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-9-maximum_rank-50.sh


In [ ]:
bash scripts/hid-1-exp-layer-12-maximum_rank-1000.sh
bash scripts/hid-1-exp-layer-12-maximum_rank-10.sh
bash scripts/hid-1-exp-layer-12-maximum_rank-300.sh

bash scripts/hid-1-exp-layer-12-maximum_rank-50.sh
bash scripts/hid-1-exp-layer-1-maximum_rank-1000.sh
bash scripts/hid-1-exp-layer-1-maximum_rank-10.sh

bash scripts/hid-1-exp-layer-1-maximum_rank-300.sh
bash scripts/hid-1-exp-layer-1-maximum_rank-50.sh
bash scripts/hid-1-exp-layer-3-maximum_rank-1000.sh

bash scripts/hid-1-exp-layer-3-maximum_rank-10.sh
bash scripts/hid-1-exp-layer-3-maximum_rank-300.sh
bash scripts/hid-1-exp-layer-3-maximum_rank-50.sh

bash scripts/hid-1-exp-layer-6-maximum_rank-1000.sh
bash scripts/hid-1-exp-layer-6-maximum_rank-10.sh
bash scripts/hid-1-exp-layer-6-maximum_rank-300.sh

bash scripts/hid-1-exp-layer-6-maximum_rank-50.sh
bash scripts/hid-1-exp-layer-9-maximum_rank-1000.sh

bash scripts/hid-1-exp-layer-9-maximum_rank-10.sh
bash scripts/hid-1-exp-layer-9-maximum_rank-300.sh

In [ ]:
bash scripts/hid-1-exp-layer-1-maximum_rank-10.sh
bash scripts/hid-1-exp-layer-3-maximum_rank-50.sh
bash scripts/hid-1-exp-layer-6-maximum_rank-50.sh

bash scripts/hid-1-exp-layer-9-maximum_rank-50.sh
bash scripts/hid-1-exp-layer-12-maximum_rank-50.sh
bash scripts/hid-1-exp-layer-12-maximum_rank-1000.sh

bash scripts/hid-1-exp-layer-3-maximum_rank-1000.sh
bash scripts/hid-1-exp-layer-6-maximum_rank-1000.sh
bash scripts/hid-1-exp-layer-9-maximum_rank-1000.sh

bash scripts/hid-1-exp-layer-9-maximum_rank-10.sh
bash scripts/hid-1-exp-layer-6-maximum_rank-10.sh
bash scripts/hid-1-exp-layer-3-maximum_rank-10.sh
bash scripts/hid-1-exp-layer-12-maximum_rank-10.sh



In [ ]:
bash scripts/hid-2-exp-layer-1-maximum_rank-300.sh
bash scripts/hid-2-exp-layer-3-maximum_rank-300.sh
bash scripts/hid-2-exp-layer-6-maximum_rank-300.sh
bash scripts/hid-2-exp-layer-9-maximum_rank-300.sh
bash scripts/hid-2-exp-layer-12-maximum_rank-300.sh


bash scripts/hid-0-exp-layer-1-maximum_rank-300.sh
bash scripts/hid-0-exp-layer-3-maximum_rank-300.sh
bash scripts/hid-0-exp-layer-6-maximum_rank-300.sh
bash scripts/hid-0-exp-layer-9-maximum_rank-300.sh
bash scripts/hid-0-exp-layer-12-maximum_rank-300.sh

scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-1-maximum_rank-300.sh
scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-3-maximum_rank-300.sh
scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-6-maximum_rank-300.sh
scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-9-maximum_rank-300.sh
scripts/do_sub_dim_rest-hid-1-exp-do_sub_dim_rest-layer-12-maximum_rank-300.sh